In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Food -> Nutrients

The item to look up in the USDA archive

In [87]:
lookup = 'ORGANIC VALLEY LOWFAT MILK'
nutrient = 'Vitamin A'

Translate into query url

In [88]:
# the query url
query_url = 'https://ndb.nal.usda.gov/ndb/search/list?SYNCHRONIZER_TOKEN=22d50fc9-d900-4447-afa8-cda5f67ea1e5&SYNCHRONIZER_URI=%2Fndb%2Fsearch%2Flist&qt=&qlookup={}&ds=&manu='.format(lookup)

Extract the NBD id of the best hit of the query

In [92]:
# open Selenium webdriver in headless mode
chrome_options = Options()
chrome_options.add_argument("headless")
driver = webdriver.Chrome(chrome_options=chrome_options)

# go to the page
driver.get(query_url)

# find the first nbd_id in the query
nbd_id = driver.find_element_by_xpath('//*[@id="quickform"]/div[4]/table/tbody/tr[1]/td[2]/a').text

Use the NBD id to get the full nutrient report from the USDA website

In [100]:
import json
api_url = "https://api.nal.usda.gov/ndb/reports/?ndbno={}&type=f&format=json&api_key=2RKrTS6ad7iBE5RapN91xfUguREfFsebWNnDer1X".format(nbd_id)
print(api_url)
# submit query
driver.get(api_url)

# get the response
soup = BeautifulSoup(driver.page_source, "lxml")
json_response = json.loads(soup.find("body").text)

https://api.nal.usda.gov/ndb/reports/?ndbno=45135747&type=f&format=json&api_key=2RKrTS6ad7iBE5RapN91xfUguREfFsebWNnDer1X


In [101]:
%%time
for ntr in json_response['report']['food']['nutrients']:
    if nutrient in ntr['name']:
        print(ntr['value'])
        if float(ntr['value']) > 0:
            print('The nutrient is present.')
        else:
            print('The nutrient is absent.')

42
The nutrient is present.
CPU times: user 512 µs, sys: 380 µs, total: 892 µs
Wall time: 586 µs
